In [2]:
# importing necessary modules

import pandas as pd # data processing, CSV file I/O
import numpy as np # linear algebra

# libraries for plotting
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Input data files are available in the './dataset' directory
import os
for dirname, _, filenames in os.walk('dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

dataset\csv1.csv
dataset\csv2.csv
dataset\merged_data.csv


In [3]:
def merge_csv(path_to_csv1: str, path_to_csv2: str):
    data1 = pd.read_csv(path_to_csv1)
    data2 = pd.read_csv(path_to_csv2)

    # Union of columns
    columns = list(set(data1.columns | data2.columns))

    d = {col: data1[col] if col in data1.columns else data2[col] for col in columns}
    merged_data = pd.DataFrame(data=d)

    # Rearranging columns
    new_order = ['DY', 'MO', 'YEAR', 'LAT', 'LON']
    columns_list = merged_data.columns.tolist()
    for col_name in new_order:
        columns_list.remove(col_name)

    columns_list = new_order + columns_list
    merged_data = merged_data[columns_list]

    return merged_data

def organize_data(merged_data : pd.DataFrame):
    organized_data = merged_data.copy()
    # If data has KT (Insolation Clearness Index) column it must be converted to float
    if 'KT' in merged_data:
        # 'KT' column may have 2 different values that describes missing values. nan and -999.0
        organized_data['KT'] = organized_data['KT'].replace('               nan', '-999')
        organized_data['KT'] = organized_data['KT'].astype(float)


    # Value for missing model data cannot be computed or out of model availability range: -999.0
    # Replace these values to np.nan in order to use pre-defined functions effectively.
    organized_data.replace(-999.0, np.nan, inplace=True)

    return organized_data

In [4]:
df = organize_data(merge_csv("dataset\csv1.csv", "dataset\csv2.csv"))

### View dimensions of dataset

In [5]:
df.shape

(2223, 28)

### Preview the dataset

In [6]:
df.head()

,DY,MO,YEAR,LAT,LON,T2M_RANGE,WS10M_MIN,WS50M_MAX,QV2M,T2M_MAX,...,WS50M,RH2M,T2MWET,TS,WS10M_RANGE,T2M_MIN,CLRSKY_SFC_SW_DWN,T2MDEW,WS50M_MIN,PS
0,1,1,2015,39.75,32.75,6.22,1.73,6.27,4.53,4.72,...,3.52,93.11,0.84,2.09,1.84,-1.51,NaN,0.83,2.14,89.50
1,2,1,2015,39.75,32.75,7.22,1.62,5.81,3.25,2.93,...,4.07,85.43,-3.51,-1.70,2.49,-4.29,NaN,-3.52,2.51,89.98
2,3,1,2015,39.75,32.75,7.97,0.36,4.69,3.24,3.41,...,2.88,88.15,-3.56,-2.49,3.57,-4.57,NaN,-3.57,0.57,89.85
3,4,1,2015,39.75,32.75,6.77,2.25,8.46,3.10,2.50,...,5.77,81.68,-4.18,-1.54,3.38,-4.27,NaN,-4.20,3.01,89.31
4,5,1,2015,39.75,32.75,5.96,2.06,8.06,3.16,2.51,...,4.76,80.07,-4.06,-0.85,4.40,-3.45,NaN,-4.08,2.93,88.72


### View column names

In [7]:
df.columns

Index(['DY', 'MO', 'YEAR', 'LAT', 'LON', 'T2M_RANGE', 'WS10M_MIN', 'WS50M_MAX',
       'QV2M', 'T2M_MAX', 'ALLSKY_SFC_LW_DWN', 'WS10M_MAX', 'PRECTOT', 'WS10M',
       'KT', 'T2M', 'ALLSKY_SFC_SW_DWN', 'WS50M_RANGE', 'WS50M', 'RH2M',
       'T2MWET', 'TS', 'WS10M_RANGE', 'T2M_MIN', 'CLRSKY_SFC_SW_DWN', 'T2MDEW',
       'WS50M_MIN', 'PS'],
      dtype='object')

### View summary of dataset 

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2223 entries, 0 to 2222
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   DY                 2223 non-null   int64  
 1   MO                 2223 non-null   int64  
 2   YEAR               2223 non-null   int64  
 3   LAT                2223 non-null   float64
 4   LON                2223 non-null   float64
 5   T2M_RANGE          2223 non-null   float64
 6   WS10M_MIN          2223 non-null   float64
 7   WS50M_MAX          2223 non-null   float64
 8   QV2M               2223 non-null   float64
 9   T2M_MAX            2223 non-null   float64
 10  ALLSKY_SFC_LW_DWN  2221 non-null   float64
 11  WS10M_MAX          2223 non-null   float64
 12  PRECTOT            2223 non-null   float64
 13  WS10M              2223 non-null   float64
 14  KT                 2213 non-null   float64
 15  T2M                2223 non-null   float64
 16  ALLSKY_SFC_SW_DWN  2215 

### View statistical properties of dataset

In [9]:
df.describe()

,DY,MO,YEAR,LAT,LON,T2M_RANGE,WS10M_MIN,WS50M_MAX,QV2M,T2M_MAX,...,WS50M,RH2M,T2MWET,TS,WS10M_RANGE,T2M_MIN,CLRSKY_SFC_SW_DWN,T2MDEW,WS50M_MIN,PS
count,2223.000000,2223.000000,2223.000000,2223.00,2223.00,2223.000000,2223.000000,2223.000000,2223.000000,2223.000000,...,2223.000000,2223.000000,2223.000000,2223.000000,2223.000000,2223.000000,892.000000,2223.000000,2223.000000,2223.000000
mean,15.736392,6.444894,2017.549258,39.75,32.75,12.481651,1.607247,6.678578,5.883792,18.013536,...,4.474215,61.681957,3.433315,11.872236,3.253981,5.531948,5.861670,3.421287,2.398106,89.672785
std,8.805415,3.486311,1.745556,0.00,0.00,3.607907,0.999735,2.223797,2.349482,10.239055,...,1.725476,18.028254,5.895196,10.079966,1.432924,7.881937,1.947353,5.977396,1.642869,0.453013
min,1.000000,1.000000,2015.000000,39.75,32.75,2.210000,0.020000,1.390000,1.230000,-7.090000,...,0.780000,14.050000,-14.390000,-14.640000,0.720000,-17.460000,2.490000,-14.840000,0.030000,87.790000
25%,8.000000,3.000000,2016.000000,39.75,32.75,9.700000,0.800000,5.140000,3.990000,9.470000,...,3.245000,47.410000,-0.865000,3.150000,2.270000,-1.020000,4.000000,-0.920000,1.050000,89.400000
50%,16.000000,6.000000,2018.000000,39.75,32.75,12.740000,1.540000,6.570000,5.490000,18.300000,...,4.240000,61.770000,3.480000,11.430000,3.020000,5.280000,6.220000,3.470000,2.150000,89.670000
75%,23.000000,9.000000,2019.000000,39.75,32.75,15.310000,2.220000,7.920000,7.700000,27.035000,...,5.400000,75.765000,8.280000,20.945000,3.905000,12.620000,7.660000,8.370000,3.385000,89.960000
max,31.000000,12.000000,2021.000000,39.75,32.75,21.750000,7.570000,18.400000,12.900000,37.990000,...,14.290000,100.000000,16.200000,30.990000,12.810000,21.900000,8.630000,16.200000,10.470000,91.220000


In [ ]:
today_precipitation = (df['PRECTOT'] > 0).astype(int)
tomorrow_precipitation = [1 if x > 0 else 0 for x in today_precipitation]
tomorrow_precipitation.pop(0)
tomorrow_precipitation.append(1)

df['PRECTOT_TODAY'] = today_precipitation
df['PRECTOT_TOMORROW'] = np.array(tomorrow_precipitation)
df.drop(columns=['LAT', 'LON'], inplace=True)

In [11]:
(df['PRECTOT'] > 0).astype(int)

0       1
1       1
2       1
3       1
4       1
       ..
2218    1
2219    1
2220    1
2221    1
2222    1
Name: PRECTOT, Length: 2223, dtype: int32

In [ ]:
df.info()